In [1]:
from keras.applications import vgg16, resnet
from keras.layers import Dense, Flatten
from keras.models import Model

# n_chan = 2
# model = vgg16.VGG16(input_shape=[201, 201, n_chan], classes=17, weights=None)
# model = resnet.ResNet50(input_shape=[201, 201, n_chan], classes=17, weights=None)

n_chan = 3
model_pre = vgg16.VGG16(input_shape=[201, 201, n_chan], include_top=False, weights='imagenet')
for layer in model_pre.layers:
    layer.trainable = False
layer = model_pre.output
layer = Flatten()(layer)
layer = Dense(4096, activation="relu")(layer)
layer = Dense(4096, activation="relu")(layer)
new_output = Dense(17, activation="softmax")(layer)
model = Model(inputs=model_pre.input, outputs=new_output)

model.summary()

In [2]:
import tensorflow as tf
import keras.backend as K
def f1(actual, predicted):
    predicted = tf.one_hot(tf.math.argmax(predicted, axis=1), depth=17)
    TP = tf.math.count_nonzero(predicted * actual)
    TN = tf.math.count_nonzero((predicted - 1) * (actual - 1))
    FP = tf.math.count_nonzero(predicted * (actual - 1))
    FN = tf.math.count_nonzero((predicted - 1) * actual)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    return f1

In [3]:
from keras.optimizers import Adam
from sklearn.metrics import f1_score
model.compile(loss='categorical_crossentropy', metrics=[f1], optimizer=Adam(lr=1e-3))

In [4]:
from dataGenerator import DataGenerator, TestDataGenerator
from personal_data import location_data
dataGenerator_train = DataGenerator(location_data, n_channels=n_chan, batch_size=128, normalize_data=2)
dataGenerator_val = DataGenerator(location_data, n_channels=n_chan, batch_size=128, normalize_data=2, isTrainingData=False)
dataGenerator_test = TestDataGenerator('../validation', n_channels=n_chan, batch_size=128, normalize_data=2)

In [5]:
from keras import callbacks
import time

# tensorboard --logdir ~/cytodata2019/tensorboard &
tensorboard = callbacks.TensorBoard(update_freq='batch',
    log_dir='../tensorboard/' + time.strftime("%Y%m%d-%H%M%S"))

In [6]:
model.fit_generator(dataGenerator_train.generate(),
    validation_data=dataGenerator_val.generate(),
    steps_per_epoch=len(dataGenerator_train),
    validation_steps=len(dataGenerator_val),
    callbacks=[tensorboard], verbose=1, epochs=100)

In [0]:
predictions = model.predict_generator(dataGenerator_test.generate(), steps=len(dataGenerator_test))

In [0]:
import numpy as np
hit_indices = np.argmax(predictions, axis=1).astype(dtype=np.int)
hit_names = [dataGenerator_train.labelsNames[i] for i in hit_indices]

In [0]:
cell_names = [s[-15:-5] for s in dataGenerator_test.allData]

In [0]:
print(len(cell_names), len(hit_names))
with open('output.csv', 'w') as file:
    file.write("cell_code,prediction\n")
    for (c,h) in zip(cell_names, hit_names):
        file.write(f"{c},{h}\n")

In [0]:
import numpy as np
(img1, type1) = next(dataGenerator_train.generate())
(img2, type2) = next(dataGenerator_train.generate())
class1 = np.argmax(type1, axis=1)

In [0]:
import matplotlib.pyplot as plt
imgsubset1 = img1[class1==5]
plt.imshow(imgsubset1[1,:,:,1])